### Import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.select import Select
import time
from tqdm.notebook import tqdm
from glob import glob
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import json
import folium
import warnings
import numpy as np


/tmp/ipykernel_13704/2647609528.py:18: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### url 접속

In [2]:
url = "https://www.opinet.co.kr/searRgSelect.do"
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.get(url)

### 추가 접속

In [3]:
driver.get(url)

### 시/도 데이터 가져오기

In [3]:
sido = driver.find_element(By.ID, "SIDO_NM0")
sido_list = sido.find_elements(By.TAG_NAME, "option")
for idx, sido_name in enumerate(sido_list):
    print(str(idx) + ". " + sido_name.get_attribute("value"))

0. 
1. 서울특별시
2. 부산광역시
3. 대구광역시
4. 인천광역시
5. 광주광역시
6. 대전광역시
7. 울산광역시
8. 세종특별자치시
9. 경기도
10. 강원특별자치도
11. 충청북도
12. 충청남도
13. 전북특별자치도
14. 전라남도
15. 경상북도
16. 경상남도
17. 제주특별자치도


### 시/도 정보 가져오고 리스트 만들기

In [4]:
sido_names = [sido_name.get_attribute("value") for sido_name in sido_list]
sido_names

['',
 '서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

### 시/도 리스트 정리

In [5]:
sido_names = sido_names[1:]
sido_names

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

### 서울시 선택 및 시/군/구 데이터 리스트로 만들기

In [6]:
sido.send_keys(sido_names[0])
gu = driver.find_element(By.ID, "SIGUNGU_NM0")
gu_list = gu.find_elements(By.TAG_NAME, "option")

gu_names = [gu_name.get_attribute("value") for gu_name in gu_list]
print(len(gu_names), gu_names)

26 ['', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


### 시/군/구 리스트 정리

In [7]:
gu_names.remove('')
print(len(gu_names), gu_names)

25 ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


In [8]:
gu.click()
gu.send_keys(gu_names[1])

### 보통 휘발유 탭에서 주유소명 클릭

In [9]:
driver.find_element(By.CSS_SELECTOR, "a[href*='javascript:fn_osPop']").click()

### 한 구의 모든 주유소 한 번씩 클릭 

In [10]:
gu_gas_stations = driver.find_elements(By.CSS_SELECTOR, "a[href*='javascript:fn_osPop']")

for gu_gas_station in gu_gas_stations:
    try:
        gu_gas_station.click()
        time.sleep(0.3)
    except ElementNotInteractableException:
        break  # 클릭할 수 없는 요소를 만나면 반복문 종료

### 메모

##### 1. 보통 휘발유 탭 상의 모든 주유소는 클릭 될 시 상세 정보가 생김
##### 2. 주유소 명 -> id="os_nm"
##### 3. 기본정보
|정보명|기본정보ID|
|:---|:---|
|전화번호|id="phn_no"|
|주소|id="rd_addr"|
|상표|id="poll_div_nm"|
##### 4. 유가 정보 ID
|유종|유종가격ID|
|:---|:---|
|고급휘발유|id="b034_p"|
|보통휘발유|id="b027_p"|
|경유|id="d047_p"|
|실내등유|id="c004_p"|
##### 5. 부가 정보 ID -> img 태그의 src=[]안의 .gif 앞에 off 유무에 따라 서비스 제공X, O
|정보명|부가정보ID|
|:---|:---|
|세차장|id="cwsh_yn"|
|충전소|id="lpg_yn"|
|경정비|id="maint_yn"|
|편의점|id="cvs_yn"|
|24시영업|id="sel24_yn"|

### 한 군/구 내의 주유소 정보 수집

In [11]:
name = driver.find_element(By.ID, "os_nm").text
phone = driver.find_element(By.ID, "phn_no").text
address = driver.find_element(By.ID, "rd_addr").text
company_name = driver.find_element(By.ID, "poll_div_nm").text
premium_gasoline = driver.find_element(By.ID, "b034_p").text
gasoline = driver.find_element(By.ID, "b027_p").text
diesel = driver.find_element(By.ID, "d047_p").text
lamp_oil = driver.find_element(By.ID, "c004_p").text
car_wash = driver.find_element(By.ID, "cwsh_yn").get_attribute("src")
charge_station = driver.find_element(By.ID, "lpg_yn").get_attribute("src")
maintain = driver.find_element(By.ID, "maint_yn").get_attribute("src")
convenience_store = driver.find_element(By.ID, "cvs_yn").get_attribute("src")
open_24h = driver.find_element(By.ID, "sel24_yn").get_attribute("src")

print(name, phone,address, company_name)
print(premium_gasoline, gasoline, diesel,lamp_oil)
print(car_wash,"\n", charge_station,"\n", maintain,"\n", convenience_store,"\n", open_24h) 



광성주유소 02-470-5133 서울 강동구 올림픽로 673 (천호동) S-OIL
 1,968 1,858 1,800
https://www.opinet.co.kr/images/user/gis/oil_station_service1_01.gif 
 https://www.opinet.co.kr/images/user/gis/oil_station_service1_02_01_off.gif 
 https://www.opinet.co.kr/images/user/gis/oil_station_service1_03.gif 
 https://www.opinet.co.kr/images/user/gis/oil_station_service1_04_off.gif 
 https://www.opinet.co.kr/images/user/gis/oil_station_service1_05.gif


### 유종 없음 표시 및 가격 정수값 변환

In [12]:
def check_empty(text):
    return text if text else "X"

def extract_integer(text):
    return int(text.replace(",", ""))

def process_fuel(text):
    if text:
        return extract_integer(text)
    else:
        return "X"

In [13]:
premium_gasoline = process_fuel(driver.find_element(By.ID, "b034_p").text)
gasoline = process_fuel(driver.find_element(By.ID, "b027_p").text)
diesel = process_fuel(driver.find_element(By.ID, "d047_p").text)
lamp_oil = process_fuel(driver.find_element(By.ID, "c004_p").text)

print(premium_gasoline, gasoline, diesel,lamp_oil)

X 1968 1858 1800


### 부가정보 O, X 반환 

In [14]:
def process_additional_info(src):
    return "O" if "off" not in src else "X"

In [15]:
car_wash = process_additional_info(driver.find_element(By.ID, "cwsh_yn").get_attribute("src"))
charge_station = process_additional_info(driver.find_element(By.ID, "lpg_yn").get_attribute("src"))
maintain = process_additional_info(driver.find_element(By.ID, "maint_yn").get_attribute("src"))
convenience_store = process_additional_info(driver.find_element(By.ID, "cvs_yn").get_attribute("src"))
open_24h = process_additional_info(driver.find_element(By.ID, "sel24_yn").get_attribute("src"))

print(car_wash, charge_station, maintain, convenience_store, open_24h)

O X O X O


### 한 구의 주유소 정보 수집 함수

In [24]:
def collect_gas_station_info():
    def process_fuel(text):
        return extract_integer(text) if text else "X"
    
    # 부가정보 O,X 반환 함수
    def process_additional_info(src):
        return "O" if "off" not in src else "X"
    
    gu_oil_station_info = {
        "이름": [],
        "전화번호": [],
        "주소": [],
        "상표": [],
        "고급휘발유": [],
        "보통휘발유": [],
        "경유": [],
        "실내등유": [],
        "세차장": [],
        "충전소": [],
        "경정비": [],
        "편의점": [],
        "24시영업": []
    }
    
    gu_gas_stations = driver.find_elements(By.CSS_SELECTOR, "a[href*='javascript:fn_osPop']")

    for gu_gas_station in gu_gas_stations:
        try:
            gu_gas_station.click()
            
            time.sleep(0.2)
            
            # 이름 수집
            name = driver.find_element(By.ID, "os_nm").text
            gu_oil_station_info["이름"].append(name)

            # 기본 정보 수집
            phone_number = check_empty(driver.find_element(By.ID, "phn_no").text)
            gu_oil_station_info["전화번호"].append(phone_number)

            address = check_empty(driver.find_element(By.ID, "rd_addr").text)
            gu_oil_station_info["주소"].append(address)

            brand = check_empty(driver.find_element(By.ID, "poll_div_nm").text)
            gu_oil_station_info["상표"].append(brand)

            # 유가 정보 수집
            premium_gasoline = process_fuel(driver.find_element(By.ID, "b034_p").text)
            gu_oil_station_info["고급휘발유"].append(premium_gasoline)

            gasoline = process_fuel(driver.find_element(By.ID, "b027_p").text)
            gu_oil_station_info["보통휘발유"].append(gasoline)

            diesel = process_fuel(driver.find_element(By.ID, "d047_p").text)
            gu_oil_station_info["경유"].append(diesel)

            lamp_oil = process_fuel(driver.find_element(By.ID, "c004_p").text)
            gu_oil_station_info["실내등유"].append(lamp_oil)

            # 부가 정보 수집
            car_wash = process_additional_info(driver.find_element(By.ID, "cwsh_yn").get_attribute("src"))
            gu_oil_station_info["세차장"].append(car_wash)

            charge_station = process_additional_info(driver.find_element(By.ID, "lpg_yn").get_attribute("src"))
            gu_oil_station_info["충전소"].append(charge_station)

            maintain = process_additional_info(driver.find_element(By.ID, "maint_yn").get_attribute("src"))
            gu_oil_station_info["경정비"].append(maintain)

            convenience_store = process_additional_info(driver.find_element(By.ID, "cvs_yn").get_attribute("src"))
            gu_oil_station_info["편의점"].append(convenience_store)

            open_24h = process_additional_info(driver.find_element(By.ID, "sel24_yn").get_attribute("src"))
            gu_oil_station_info["24시영업"].append(open_24h)

        except ElementNotInteractableException:
            break  # 클릭할 수 없는 요소를 만나면 반복문 종료

    return gu_oil_station_info

### 모든 구의 주유소 정보

In [25]:
all_gu_oil_station_info = {}
for gu_name in tqdm(gu_names):
    select = Select(driver.find_element(By.ID, "SIGUNGU_NM0"))
    select.select_by_visible_text(gu_name)
    all_gu_oil_station_info[gu_name] = collect_gas_station_info()

  0%|          | 0/25 [00:00<?, ?it/s]

### 결과 출력

In [26]:
# all_gu_oil_station_info를 데이터프레임으로 변환
df_list = []
for gu_name, station_info in all_gu_oil_station_info.items():
    df = pd.DataFrame(station_info)
    df["구 이름"] = gu_name  # 각 주유소 정보에 '구 이름' 열 추가
    df_list.append(df)

# 데이터프레임들을 하나의 데이터프레임으로 결합
all_gu_oil_station_df = pd.concat(df_list, ignore_index=True)

# 결과 출력
print(all_gu_oil_station_df)

    이름          전화번호                     주소        상표 고급휘발유 보통휘발유    경유  실내등유  \
0                  X                      X         X     X  1655  1553     X   
1       02-2226-4963         서울 강남구 헌릉로 730  HD현대오일뱅크  1859  1665  1565     X   
2                  X                      X         X     X     X     X     X   
3                  X                      X         X  1875  1685  1595     X   
4        02-459-3434  서울 강남구 밤고개로 215 (율현동)     GS칼텍스     X  1687  1589     X   
..  ..           ...                    ...       ...   ...   ...   ...   ...   
430     02-2207-9419  서울 중랑구 용마산로 716 (신내동)     S-OIL     X  1639  1539  1450   
431     02-2207-0523  서울 중랑구 용마산로 705 (신내동)     SK에너지     X  1652  1553  1450   
432                X                      X         X     X  1658  1538  1600   
433      02-436-3651    서울 중랑구 상봉로 58 (망우동)     SK에너지     X  1659  1568  1590   
434      02-974-8356    서울 중랑구 동일로 881 (묵동)     S-OIL     X  1698  1598  1600   

    세차장 충전소 경정비 편의점 24시영업 구

In [27]:
all_gu_oil_station_df.to_csv("../data/gas_station_ver1")
all_gu_oil_station_df.to_excel("../data/gas_station_ver1.xlsx")

### 위 결과를 확인해보면 정보 누락이 된 결과가 나타남. 
#### selenium 크롬 창 확인 결과 스크롤이 덜 되어 보이지 않는 정보가 누락됨을 확인. -> 항상 보이는 요소와 팝업 요소의 좌표를 일치시켜 해결 시도

#### 좌표 구하는 코드

In [17]:
# 클래스 이름을 사용하여 팝업 요소를 찾기
popup = driver.find_element(By.CLASS_NAME, "ollehmap-info-defaultStyle")

# 요소의 위치 정보 가져오기
location_popup = popup.location

# X 및 Y 좌표 출력
print("X 좌표:", location_popup['x'])
print("Y 좌표:", location_popup['y'])

X 좌표: 281
Y 좌표: 84


In [18]:
# 클래스 이름을 사용하여 항상 보이는 요소를 찾기
panright = driver.find_element(By.CLASS_NAME, "control-panright")

# 요소의 위치 정보 가져오기
location_panright = panright.location

# X 및 Y 좌표 출력
print("X 좌표:", location_panright['x'])
print("Y 좌표:", location_panright['y'])

X 좌표: 389
Y 좌표: 108


#### class=control-pan(up, down, left, right) 요소 클릭 -> 스크롤

In [19]:
scroll_up = driver.find_element(By.CLASS_NAME, "control-pandown")
ActionChains(driver).click_and_hold(scroll_up).perform()
time.sleep(0.0001)
ActionChains(driver).release(scroll_up).perform()

In [29]:
# 예시 코드 중에서 사용하는 팝업 좌표와 타겟 좌표
popup_x = location_popup['x']
popup_y = location_popup['y']
target_x = location_panright['x']
target_y = location_panright['y']
error_range = 55

# 팝업 좌표와 화면 정중앙 좌표 비교하여 스크롤하기
while True:
        # name 요소가 존재할 경우 루프 탈출
        try:
            name_element = driver.find_element(By.ID, "os_nm")
            if name_element.text:
                break
        except NoSuchElementException:
            pass

        x_difference = abs(popup_x - target_x)
        y_difference = abs(popup_y - target_y)

        # x 좌표의 차이가 오차 범위 내에 있는지 확인하여 스크롤 결정
        if x_difference > error_range:
            # Determine scroll direction
            scroll_direction = "left" if popup_x < target_x else "right"
            scroll_element = driver.find_element(By.CLASS_NAME, f"control-pan{scroll_direction}")
            ActionChains(driver).click_and_hold(scroll_element).perform()
            time.sleep(0.015)
            ActionChains(driver).release(scroll_element).perform()  # 홀드 해제


        # y 좌표의 차이가 오차 범위 내에 있는지 확인하여 스크롤 결정
        if y_difference > error_range:
            # Determine scroll direction
            scroll_direction = "up" if popup_y < target_y else "down"
            scroll_element = driver.find_element(By.CLASS_NAME, f"control-pan{scroll_direction}")
            ActionChains(driver).click_and_hold(scroll_element).perform()
            time.sleep(0.0001)
            ActionChains(driver).release(scroll_element).perform()  # 홀드 해제
            # name 요소가 존재할 경우 루프 탈출
            try:
                name_element = driver.find_element(By.ID, "os_nm")
                if name_element.text:
                    break
            except NoSuchElementException:
                pass

            while -18 < driver.find_element(By.CLASS_NAME, "ollehmap-info-defaultStyle").location['y'] < 50:
                ActionChains(driver).double_click(scroll_element).perform()

        # name 요소가 존재할 경우 루프 탈출
        try:
            name_element = driver.find_element(By.ID, "os_nm")
            if name_element.text:
                break
        except NoSuchElementException:
            pass

        # 스크롤 후 팝업 좌표 갱신
        element = driver.find_element(By.CLASS_NAME, "ollehmap-info-defaultStyle")
        location = element.location
        popup_x = location['x']
        popup_y = location['y']

        # 더 이상 스크롤할 필요가 없는 경우 루프 탈출
        if x_difference <= error_range and y_difference <= error_range:
            break

### 스크롤 함수

In [30]:
def scroll_to_align(target_x, target_y, error_range=55):
    # 예시 코드 중에서 사용하는 팝업 좌표와 타겟 좌표
    popup_x = location_popup['x']
    popup_y = location_popup['y']

    while True:
        # name 요소가 존재할 경우 루프 탈출
        try:
            name_element = driver.find_element(By.ID, "os_nm")
            if name_element.text:
                break
        except NoSuchElementException:
            pass

        x_difference = abs(popup_x - target_x)
        y_difference = abs(popup_y - target_y)

        # x 좌표의 차이가 오차 범위 내에 있는지 확인하여 스크롤 결정
        if x_difference > error_range:
            # Determine scroll direction
            scroll_direction = "left" if popup_x < target_x else "right"
            scroll_element = driver.find_element(By.CLASS_NAME, f"control-pan{scroll_direction}")
            ActionChains(driver).click_and_hold(scroll_element).perform()
            time.sleep(0.015)
            ActionChains(driver).release(scroll_element).perform()  # 홀드 해제


        # y 좌표의 차이가 오차 범위 내에 있는지 확인하여 스크롤 결정
        if y_difference > error_range:
            # Determine scroll direction
            scroll_direction = "up" if popup_y < target_y else "down"
            scroll_element = driver.find_element(By.CLASS_NAME, f"control-pan{scroll_direction}")
            ActionChains(driver).click_and_hold(scroll_element).perform()
            time.sleep(0.0001)
            ActionChains(driver).release(scroll_element).perform()  # 홀드 해제
            # name 요소가 존재할 경우 루프 탈출
            try:
                name_element = driver.find_element(By.ID, "os_nm")
                if name_element.text:
                    break
            except NoSuchElementException:
                pass
            
            while -15 < driver.find_element(By.CLASS_NAME, "ollehmap-info-defaultStyle").location['y'] < 50:
                    ActionChains(driver).double_click(scroll_element).perform()

        # name 요소가 존재할 경우 루프 탈출
        try:
            name_element = driver.find_element(By.ID, "os_nm")
            if name_element.text:
                break
        except NoSuchElementException:
            pass

        # 스크롤 후 팝업 좌표 갱신
        element = driver.find_element(By.CLASS_NAME, "ollehmap-info-defaultStyle")
        location = element.location
        popup_x = location['x']
        popup_y = location['y']

        # 더 이상 스크롤할 필요가 없는 경우 루프 탈출
        if x_difference <= error_range and y_difference <= error_range:
            break

### 정보 수집

In [31]:
def collect_gas_station_info():
    wait = WebDriverWait(driver, 0.5)

    def process_fuel(text):
        return extract_integer(text) if text else "X"
    
    def process_additional_info(src):
        return "O" if "off" not in src else "X"
    
    gu_oil_station_info = {
        "이름": [],
        "전화번호": [],
        "주소": [],
        "상표": [],
        "고급휘발유": [],
        "보통휘발유": [],
        "경유": [],
        "실내등유": [],
        "세차장": [],
        "충전소": [],
        "경정비": [],
        "편의점": [],
        "24시영업": []
    }
    
    gu_gas_stations = driver.find_elements(By.CSS_SELECTOR, "a[href*='javascript:fn_osPop']")

    for gu_gas_station in gu_gas_stations:
        try:
            gu_gas_station.click()

            # 이름 수집
            name_element = None
            try:
                name_element = wait.until(EC.visibility_of_element_located((By.ID, "os_nm")))
            except:
                pass
            
            if name_element:
                name = name_element.text
            else:
                # 텍스트가 없거나 None인 경우 대체 텍스트 수집을 위해 다른 요소 클릭
                scroll_to_align(target_x, target_y)
                name_element = wait.until(EC.visibility_of_element_located((By.ID, "os_nm")))
                name = name_element.text if name_element else "Unknown"
                
            gu_oil_station_info["이름"].append(name)

            # 기본 정보 수집
            phone_number = check_empty(driver.find_element(By.ID, "phn_no").text)
            gu_oil_station_info["전화번호"].append(phone_number)

            address = check_empty(driver.find_element(By.ID, "rd_addr").text)
            gu_oil_station_info["주소"].append(address)

            brand = check_empty(driver.find_element(By.ID, "poll_div_nm").text)
            gu_oil_station_info["상표"].append(brand)

            # 유가 정보 수집
            premium_gasoline = process_fuel(driver.find_element(By.ID, "b034_p").text)
            gu_oil_station_info["고급휘발유"].append(premium_gasoline)

            gasoline = process_fuel(driver.find_element(By.ID, "b027_p").text)
            gu_oil_station_info["보통휘발유"].append(gasoline)

            diesel = process_fuel(driver.find_element(By.ID, "d047_p").text)
            gu_oil_station_info["경유"].append(diesel)

            lamp_oil = process_fuel(driver.find_element(By.ID, "c004_p").text)
            gu_oil_station_info["실내등유"].append(lamp_oil)

            # 부가 정보 수집
            car_wash = process_additional_info(driver.find_element(By.ID, "cwsh_yn").get_attribute("src"))
            gu_oil_station_info["세차장"].append(car_wash)

            charge_station = process_additional_info(driver.find_element(By.ID, "lpg_yn").get_attribute("src"))
            gu_oil_station_info["충전소"].append(charge_station)

            maintain = process_additional_info(driver.find_element(By.ID, "maint_yn").get_attribute("src"))
            gu_oil_station_info["경정비"].append(maintain)

            convenience_store = process_additional_info(driver.find_element(By.ID, "cvs_yn").get_attribute("src"))
            gu_oil_station_info["편의점"].append(convenience_store)

            open_24h = process_additional_info(driver.find_element(By.ID, "sel24_yn").get_attribute("src"))
            gu_oil_station_info["24시영업"].append(open_24h)

        except ElementNotInteractableException:
            break  # 클릭할 수 없는 요소를 만나면 반복문 종료

    return gu_oil_station_info

### 모든 구의 주유소 정보 수집

In [32]:
all_gu_oil_station_info = {}
for gu_name in tqdm(gu_names):
    select = Select(driver.find_element(By.ID, "SIGUNGU_NM0"))
    select.select_by_visible_text(gu_name)
    all_gu_oil_station_info[gu_name] = collect_gas_station_info()

  0%|          | 0/25 [00:00<?, ?it/s]

In [33]:
# all_gu_oil_station_info를 데이터프레임으로 변환
df_list = []
for gu_name, station_info in all_gu_oil_station_info.items():
    df = pd.DataFrame(station_info)
    df["구 이름"] = gu_name  # 각 주유소 정보에 '구 이름' 열 추가
    df_list.append(df)

# 데이터프레임들을 하나의 데이터프레임으로 결합
all_gu_oil_station_df = pd.concat(df_list, ignore_index=True)

# 결과 출력
print(all_gu_oil_station_df)

                      이름          전화번호                       주소        상표  \
0            (주)보성 세곡주유소   02-445-6870     서울 강남구 헌릉로 731 (세곡동)     SK에너지   
1    HD현대오일뱅크㈜직영 산성셀프주유소  02-2226-4963           서울 강남구 헌릉로 730  HD현대오일뱅크   
2               오일프러스 셀프  02-3462-5100  서울 강남구 남부순환로 2651 (도곡동)     SK에너지   
3           극동유화㈜ 개나리주유소   02-564-0186     서울 강남구 언주로 423 (역삼동)     S-OIL   
4                  방죽주유소   02-459-3434    서울 강남구 밤고개로 215 (율현동)     GS칼텍스   
..                   ...           ...                      ...       ...   
431             (주)기지에너지  02-2207-9419    서울 중랑구 용마산로 716 (신내동)     S-OIL   
432                신내주유소  02-2207-0523    서울 중랑구 용마산로 705 (신내동)     SK에너지   
433               용마로주유소   02-439-3037    서울 중랑구 용마산로 309 (면목동)     SK에너지   
434              신일셀프주유소   02-436-3651      서울 중랑구 상봉로 58 (망우동)     SK에너지   
435                범아주유소   02-974-8356      서울 중랑구 동일로 881 (묵동)     S-OIL   

    고급휘발유  보통휘발유    경유  실내등유 세차장 충전소 경정비 편의점 24시영업 구 이름  
0       X   1655 

In [34]:
all_gu_oil_station_df.to_csv("../data/gas_station_ver2")
all_gu_oil_station_df.to_excel("../data/gas_station_ver2.xlsx")

In [35]:
driver.close();

In [36]:
raw_df = pd.read_excel("../data/gas_station_ver2.xlsx")
print(raw_df)

     Unnamed: 0                   이름          전화번호                       주소  \
0             0          (주)보성 세곡주유소   02-445-6870     서울 강남구 헌릉로 731 (세곡동)   
1             1  HD현대오일뱅크㈜직영 산성셀프주유소  02-2226-4963           서울 강남구 헌릉로 730   
2             2             오일프러스 셀프  02-3462-5100  서울 강남구 남부순환로 2651 (도곡동)   
3             3         극동유화㈜ 개나리주유소   02-564-0186     서울 강남구 언주로 423 (역삼동)   
4             4                방죽주유소   02-459-3434    서울 강남구 밤고개로 215 (율현동)   
..          ...                  ...           ...                      ...   
431         431             (주)기지에너지  02-2207-9419    서울 중랑구 용마산로 716 (신내동)   
432         432                신내주유소  02-2207-0523    서울 중랑구 용마산로 705 (신내동)   
433         433               용마로주유소   02-439-3037    서울 중랑구 용마산로 309 (면목동)   
434         434              신일셀프주유소   02-436-3651      서울 중랑구 상봉로 58 (망우동)   
435         435                범아주유소   02-974-8356      서울 중랑구 동일로 881 (묵동)   

           상표 고급휘발유  보통휘발유    경유  실내등유 세차장 충전소 경정비 

In [56]:
sorted_df_by_gasoline = df.sort_values(by='보통휘발유', ascending=True)
print(sorted_df_by_gasoline)
sorted_df_by_gasoline.to_excel("../data/gas_station_sorted_by_gasoline.xlsx")

                 이름           전화번호                       주소        상표 고급휘발유  \
0             우림주유소    02-433-9990  서울 중랑구 용마산로 487 (망우제3동)     GS칼텍스     X   
1    (주)자연에너지 대창주유소    02-434-1448     서울 중랑구 동일로 636 (면목동)  HD현대오일뱅크  1830   
2             대원주유소    02-438-6111           서울 중랑구 동일로 600     GS칼텍스  1830   
3         면목SELF주유소    02-437-4072     서울 중랑구 동일로 627 (면목동)     SK에너지     X   
4            오천만주유소  070-7797-7474     서울 중랑구 동일로 547 (면목동)     S-OIL     X   
5         구도일주유소 동천    02-495-0081     서울 중랑구 동일로 654 (면목동)     S-OIL     X   
6             대양주유소    02-437-5457    서울 중랑구 봉우재로 105 (상봉동)     GS칼텍스  1840   
7   (주)태영 구도일주유소 한인    02-438-5151     서울 중랑구 망우로 170 (상봉동)     S-OIL  1845   
8          (주)기지에너지   02-2207-9419    서울 중랑구 용마산로 716 (신내동)     S-OIL     X   
9             신내주유소   02-2207-0523    서울 중랑구 용마산로 705 (신내동)     SK에너지     X   
10           용마로주유소    02-439-3037    서울 중랑구 용마산로 309 (면목동)     SK에너지     X   
11          신일셀프주유소    02-436-3651      서울 중랑구 상봉로 5

In [38]:
geo_path = "../data/02. skorea_municipalities_geo_simple.json"
geo_str = json.load(open(geo_path, encoding="utf-8"))

In [61]:
# 각 구 이름에 따른 보통 휘발유 가격의 평균 계산
avg_gasoline_price_by_district = raw_df.groupby("구 이름")["보통휘발유"].mean()

# 지도 생성
my_map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

# Choropleth를 사용하여 평균 보통 휘발유 가격을 지도에 표시
folium.Choropleth(
    geo_data=geo_str,
    data=avg_gasoline_price_by_district,
    columns=[avg_gasoline_price_by_district.index, avg_gasoline_price_by_district.values],
    key_on="feature.id",  # Corrected key
    fill_color="PuRd",
    fill_opacity=0.7,
    legend_name="평균 보통휘발유 가격"
).add_to(my_map)

# 지도 출력
my_map

In [69]:
geo_local = Nominatim(user_agent='South Korea')

In [70]:
# 주소를 이용하여 위도, 경도를 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        if geo:
            x_y = [geo.latitude, geo.longitude]
            return x_y
        else:
            return [0, 0]  # 주소 정보를 찾을 수 없는 경우
    except:
        return [0, 0]  # 예외 발생 시

In [72]:
# 주유소 주소를 위도, 경도로 변환하여 리스트에 저장
latitude = []
longitude = []

for 주소 in df['주소']:
    좌표 = geocoding(주소)
    latitude.append(좌표[0])
    longitude.append(좌표[1])

# 위도와 경도가 포함된 데이터프레임 생성
df['위도'] = latitude
df['경도'] = longitude

In [73]:
# 서울 중심에 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 각 주유소 위치에 마커 추가
for index, row in df.iterrows():
    folium.Marker([row['위도'], row['경도']], popup=row['이름']).add_to(seoul_map)

# 지도 표시
seoul_map